In [8]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import os
import pandas as pd
import ccxt

In [9]:
# Import environment variables
kraken_public_key = os.getenv('KRAKEN_PUBLIC_KEY')
kraken_secret_key = os.getenv('KRAKEN_SECRET_KEY')
COIN_MARKET_KEY = os.getenv('COIN_MARKET_KEY')

# Set the public and private keys for the KRAKEN API
exchange = ccxt.kraken({
    'apiKey': kraken_public_key,
    'secret': kraken_secret_key,
})

In [10]:
# Connect to CoinMarketCap and load the available cryptocurrencies
url = 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': COIN_MARKET_KEY,
}
session = Session()
session.headers.update(headers)
try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
#   print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [11]:
# Import data as a Pandas DataFrame
crypto_df = pd.DataFrame.from_dict(data['data'])

In [12]:
crypto_df[['USD']] = pd.DataFrame(crypto_df.quote.values.tolist())
crypto_df[['Price', 'Volume_24h', 'Percent_change_1h', 'Percent_change_24h', 'Percent_change_7d', 'Market_cap', 'Last_updated']] = pd.DataFrame(crypto_df.USD.values.tolist())

In [13]:
crypto_df.drop(crypto_df.columns.difference(['cmc_rank', 'name', 'symbol', 'tags', 'Price', 'circulating_supply', 'total_supply', 'Volume_24h', 'Market_cap', 'max_supply']), 1, inplace=False)
crypto_df = crypto_df[['cmc_rank', 'name', 'symbol', 'tags', 'Price', 'circulating_supply', 'total_supply', 'max_supply', 'Volume_24h', 'Market_cap']]

In [14]:
crypto_df_20 = crypto_df.sort_values(by='Volume_24h', ascending=False)
crypto_df_20.head()

,cmc_rank,name,symbol,tags,Price,circulating_supply,total_supply,max_supply,Volume_24h,Market_cap
0,1,Bitcoin,BTC,[mineable],2019-08-30T18:51:28.000Z,1.790601e+07,1.790601e+07,2.100000e+07,1.711555e+11,9558.551637
1,2,Ethereum,ETH,[mineable],2019-08-30T18:51:21.000Z,1.075379e+08,1.075379e+08,NaN,1.814043e+10,168.688634
2,3,XRP,XRP,[],2019-08-30T18:51:03.000Z,4.293287e+10,9.999137e+10,1.000000e+11,1.092420e+10,0.254449
3,4,Bitcoin Cash,BCH,[mineable],2019-08-30T18:51:08.000Z,1.797598e+07,1.797598e+07,2.100000e+07,5.051770e+09,281.029003
4,5,Litecoin,LTC,[mineable],2019-08-30T18:51:04.000Z,6.314712e+07,6.314712e+07,8.400000e+07,4.063460e+09,64.349090


In [16]:
# Connect to Kraken and load the available cryptocurrencies
crypto_details = exchange.load_markets()

# Import data as a Pandas DataFrame
crypto_details_1 = pd.DataFrame(crypto_details)
crypto_details_1

,ADA/ETH,ADA/EUR,ADA/USD,ADA/BTC,ALGO/ETH,ALGO/EUR,ALGO/USD,ALGO/BTC,ATOM/ETH,ATOM/EUR,...,XRP/EUR,XRP/JPY,XRP/USD,ZEC/BTC,ZEC/EUR,ZEC/USD,EUR/USD,GBP/USD,USD/CAD,USD/JPY
active,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
altname,ADAETH,ADAEUR,ADAUSD,ADAXBT,ALGOETH,ALGOEUR,ALGOUSD,ALGOXBT,ATOMETH,ATOMEUR,...,XRPEUR,XRPJPY,XRPUSD,ZECXBT,ZECEUR,ZECUSD,EURUSD,GBPUSD,USDCAD,USDJPY
base,ADA,ADA,ADA,ADA,ALGO,ALGO,ALGO,ALGO,ATOM,ATOM,...,XRP,XRP,XRP,ZEC,ZEC,ZEC,EUR,GBP,USD,USD
baseId,ADA,ADA,ADA,ADA,ALGO,ALGO,ALGO,ALGO,ATOM,ATOM,...,XXRP,XXRP,XXRP,XZEC,XZEC,XZEC,ZEUR,ZGBP,ZUSD,ZUSD
darkpool,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
id,ADAETH,ADAEUR,ADAUSD,ADAXBT,ALGOETH,ALGOEUR,ALGOUSD,ALGOXBT,ATOMETH,ATOMEUR,...,XXRPZEUR,XXRPZJPY,XXRPZUSD,XZECXXBT,XZECZEUR,XZECZUSD,ZEURZUSD,ZGBPZUSD,ZUSDZCAD,ZUSDZJPY
info,"{'altname': 'ADAETH', 'wsname': 'ADA/ETH', 'ac...","{'altname': 'ADAEUR', 'wsname': 'ADA/EUR', 'ac...","{'altname': 'ADAUSD', 'wsname': 'ADA/USD', 'ac...","{'altname': 'ADAXBT', 'wsname': 'ADA/XBT', 'ac...","{'altname': 'ALGOETH', 'wsname': 'ALGO/ETH', '...","{'altname': 'ALGOEUR', 'wsname': 'ALGO/EUR', '...","{'altname': 'ALGOUSD', 'wsname': 'ALGO/USD', '...","{'altname': 'ALGOXBT', 'wsname': 'ALGO/XBT', '...","{'altname': 'ATOMETH', 'wsname': 'ATOM/ETH', '...","{'altname': 'ATOMEUR', 'wsname': 'ATOM/EUR', '...",...,"{'altname': 'XRPEUR', 'wsname': 'XRP/EUR', 'ac...","{'altname': 'XRPJPY', 'wsname': 'XRP/JPY', 'ac...","{'altname': 'XRPUSD', 'wsname': 'XRP/USD', 'ac...","{'altname': 'ZECXBT', 'wsname': 'ZEC/XBT', 'ac...","{'altname': 'ZECEUR', 'wsname': 'ZEC/EUR', 'ac...","{'altname': 'ZECUSD', 'wsname': 'ZEC/USD', 'ac...","{'altname': 'EURUSD', 'wsname': 'EUR/USD', 'ac...","{'altname': 'GBPUSD', 'wsname': 'GBP/USD', 'ac...","{'altname': 'USDCAD', 'wsname': 'USD/CAD', 'ac...","{'altname': 'USDJPY', 'wsname': 'USD/JPY', 'ac..."
limits,"{'amount': {'min': 1.0, 'max': 100000000.0}, '...","{'amount': {'min': 1.0, 'max': 100000000.0}, '...","{'amount': {'min': 1.0, 'max': 100000000.0}, '...","{'amount': {'min': 1.0, 'max': 100000000.0}, '...","{'amount': {'min': 50.0, 'max': 100000000.0}, ...","{'amount': {'min': 50.0, 'max': 100000000.0}, ...","{'amount': {'min': 50.0, 'max': 100000000.0}, ...","{'amount': {'min': 50.0, 'max': 100000000.0}, ...","{'amount': {'min': 1.0, 'max': 100000000.0}, '...","{'amount': {'min': 1.0, 'max': 100000000.0}, '...",...,"{'amount': {'min': 30.0, 'max': 100000000.0}, ...","{'amount': {'min': 30.0, 'max': 100000000.0}, ...","{'amount': {'min': 30.0, 'max': 100000000.0}, ...","{'amount': {'min': 0.03, 'max': 100000000.0}, ...","{'amount': {'min': 0.03, 'max': 100000000.0}, ...","{'amount': {'min': 0.03, 'max': 100000000.0}, ...","{'amount': {'min': 10.0, 'max': 100000000.0}, ...","{'amount': {'min': 10.0, 'max': 100000000.0}, ...","{'amount': {'min': 10.0, 'max': 100000000.0}, ...","{'amount': {'min': 10.0, 'max': 100000000.0}, ..."
maker,0.0016,0.0016,0.0016,0.0016,0.0016,0.0016,0.0016,0.0016,0.0016,0.0016,...,0.0016,0.0016,0.0016,0.0016,0.0016,0.0016,0.002,0.002,0.002,0.002
percentage,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [43]:
# Fetch daily candlestick bar data from Top 5 Crypto
BTC_historical_prices = exchange.fetch_ohlcv("BTC/USD", "1d")
ETH_historical_prices = exchange.fetch_ohlcv("ETH/USD", "1d")
XRP_historical_prices = exchange.fetch_ohlcv("XRP/USD", "1d")
BCH_historical_prices = exchange.fetch_ohlcv("BCH/USD", "1d")
LTC_historical_prices = exchange.fetch_ohlcv("LTC/USD", "1d")
USDT_historical_prices = exchange.fetch_ohlcv("USDT/USD", "1d")
EOS_historical_prices = exchange.fetch_ohlcv("EOS/USD", "1d")
# BNB_historical_prices = exchange.fetch_ohlcv("BNB/USD", "1d")
# BSV_historical_prices = exchange.fetch_ohlcv("BSV/USD", "1d")
XLM_historical_prices = exchange.fetch_ohlcv("XLM/USD", "1d")

# Import the data as a Pandas DataFrame and set the columns
BTC_historical_prices_df = pd.DataFrame(
    BTC_historical_prices, columns=["timestamp", "open", "high", "low", "close", "volume"])

ETH_historical_prices_df = pd.DataFrame(
    ETH_historical_prices, columns=["timestamp", "open", "high", "low", "close", "volume"])

XRP_historical_prices_df = pd.DataFrame(
    XRP_historical_prices, columns=["timestamp", "open", "high", "low", "close", "volume"])

BCH_historical_prices_df = pd.DataFrame(
    BCH_historical_prices, columns=["timestamp", "open", "high", "low", "close", "volume"])

LTC_historical_prices_df = pd.DataFrame(
    LTC_historical_prices, columns=["timestamp", "open", "high", "low", "close", "volume"])

USDT_historical_prices_df = pd.DataFrame(
    USDT_historical_prices, columns=["timestamp", "open", "high", "low", "close", "volume"])

EOS_historical_prices_df = pd.DataFrame(
    EOS_historical_prices, columns=["timestamp", "open", "high", "low", "close", "volume"])

XLM_historical_prices_df = pd.DataFrame(
    XLM_historical_prices, columns=["timestamp", "open", "high", "low", "close", "volume"])

# Convert epoch timestamp to date using the `to_datetime` function and `unit` parameter and set date as index
BTC_historical_prices_df["date"] = pd.to_datetime(BTC_historical_prices_df["timestamp"], unit="ms")
BTC_historical_prices_df.set_index("date", inplace=True)

ETH_historical_prices_df["date"] = pd.to_datetime(ETH_historical_prices_df["timestamp"], unit="ms")
ETH_historical_prices_df.set_index("date", inplace=True)

XRP_historical_prices_df["date"] = pd.to_datetime(XRP_historical_prices_df["timestamp"], unit="ms")
XRP_historical_prices_df.set_index("date", inplace=True)

BCH_historical_prices_df["date"] = pd.to_datetime(BCH_historical_prices_df["timestamp"], unit="ms")
BCH_historical_prices_df.set_index("date", inplace=True)

LTC_historical_prices_df["date"] = pd.to_datetime(LTC_historical_prices_df["timestamp"], unit="ms")
LTC_historical_prices_df.set_index("date", inplace=True)

USDT_historical_prices_df["date"] = pd.to_datetime(USDT_historical_prices_df["timestamp"], unit="ms")
USDT_historical_prices_df.set_index("date", inplace=True)

EOS_historical_prices_df["date"] = pd.to_datetime(EOS_historical_prices_df["timestamp"], unit="ms")
EOS_historical_prices_df.set_index("date", inplace=True)

XLM_historical_prices_df["date"] = pd.to_datetime(XLM_historical_prices_df["timestamp"], unit="ms")
XLM_historical_prices_df.set_index("date", inplace=True)

In [46]:
# Drop all columns but Close pricing and Concat the Crypto dataframes
Crypto_historical_prices_df = pd.concat([
    BTC_historical_prices_df.drop(columns = ["timestamp", "open", "high", "low", "volume"], inplace = False)
    .rename(columns={"close": "BTC Closing"}),
    ETH_historical_prices_df.drop(columns = ["timestamp", "open", "high", "low", "volume"],inplace = False)
    .rename(columns={"close": "ETH Closing"}),
    XRP_historical_prices_df.drop(columns = ["timestamp", "open", "high", "low", "volume"],inplace = False)
    .rename(columns={"close": "XRP Closing"}),
    BCH_historical_prices_df.drop(columns = ["timestamp", "open", "high", "low", "volume"],inplace = False)
    .rename(columns={"close": "BCH Closing"}),
    LTC_historical_prices_df.drop(columns = ["timestamp", "open", "high", "low", "volume"],inplace = False)
    .rename(columns={"close": "LTC Closing"}),
    USDT_historical_prices_df.drop(columns = ["timestamp", "open", "high", "low", "volume"],inplace = False)
    .rename(columns={"close": "USDT Closing"}),
    EOS_historical_prices_df.drop(columns = ["timestamp", "open", "high", "low", "volume"],inplace = False)
    .rename(columns={"close": "EOS Closing"}),
    XLM_historical_prices_df.drop(columns = ["timestamp", "open", "high", "low", "volume"],inplace = False)
    .rename(columns={"close": "XLM Closing"})
], axis = 'columns', join = 'inner')

In [47]:
Crypto_historical_prices_df.head()

,BTC Closing,ETH Closing,XRP Closing,BCH Closing,LTC Closing,USDT Closing,EOS Closing,XLM Closing
date,,,,,,,,
2018-07-19,7472.3,468.45,0.47629,825.6,86.10,0.9999,8.3881,0.305290
2018-07-20,7320.0,447.95,0.44360,762.5,81.63,0.9981,7.8300,0.269272
2018-07-21,7396.7,460.88,0.45286,787.3,83.49,0.9999,8.0149,0.292800
2018-07-22,7401.1,457.34,0.44979,784.8,82.33,0.9988,7.8363,0.283689
2018-07-23,7720.0,449.38,0.44555,782.4,81.52,0.9976,7.9349,0.286605


In [53]:
# returns of crypto
Crypto_returns_df = Crypto_historical_prices_df.pct_change()

# Drop Nulls
Crypto_returns_df.dropna(inplace = True)

In [54]:
Crypto_returns_df.head()

,BTC Closing,ETH Closing,XRP Closing,BCH Closing,LTC Closing,USDT Closing,EOS Closing,XLM Closing
date,,,,,,,,
2018-07-20,-0.020382,-0.043761,-0.068635,-0.076429,-0.051916,-0.001800,-0.066535,-0.117980
2018-07-21,0.010478,0.028865,0.020875,0.032525,0.022786,0.001803,0.023614,0.087376
2018-07-22,0.000595,-0.007681,-0.006779,-0.003175,-0.013894,-0.001100,-0.022283,-0.031117
2018-07-23,0.043088,-0.017405,-0.009427,-0.003058,-0.009838,-0.001201,0.012582,0.010279
2018-07-24,0.088316,0.070653,0.027090,0.107745,0.094701,0.001002,0.083555,0.049877


In [61]:
# the function that we'll optimize is "neg_portfolio_sharpR"
def neg_portfolio_sharpR (weights):
    crypto_portfolio_df = Crypto_returns_df.dot(weights)
    # We need cumulative returns
    crypto_cum_returns_df = ( 1 + crypto_portfolio_df).cumprod()
    # Then we solve for annual_return
    crypto_returns_annual = (crypto_cum_returns_df)**(365/(b -a).days) - 1
    crypto_returns_annual = crypto_returns_annual[-1]

    crypto_returns_sd_ann = crypto_portfolio_df.std()*np.sqrt(252)
    # we'll use a minimization routine
    # so we define the Sharpe_ratio as negative
    return (-crypto_returns_annual/crypto_returns_sd_ann)

In [62]:
from scipy.optimize import minimize
import numpy as np

In [63]:
num_assets = 8
# The constraint is that the sum of weights needs to be equal to 1:
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bound = (0.0,1.0) # no negative weights (no short-selling)
bounds = tuple(bound for asset in range(num_assets))
# We use the Sequential Least SQuares Programming (SLSQP) Algorithm
result = minimize(neg_portfolio_sharpR, num_assets*[1./num_assets,],
                        method='SLSQP', bounds=bounds, constraints=constraints)

NameError: name 'b' is not defined

In [ ]:
# optimal Sharpe ratio is the negative of the minimum
optimal_Sharpe = -result.fun
optimal_Sharpe

In [ ]:
opt_weights = pd.DataFrame(result.x)
opt_weights

In [ ]:
# setting up weights in a dataframe
def opt_weights_df(results):
    opt_weights_df = pd.DataFrame(result.x).transpose()
    # renaming columns with tickers names
    opt_weights_df = pd.DataFrame (data = opt_weights_df.values, columns = returns_df.columns)
    return opt_weights_df

In [ ]:
opt_weights_df(result)

In [ ]:
# fixing the dataframe with weights in order to
# be able to plot a pie
opt_weights_transpose = opt_weights_df(result).T
opt_weights_transpose['weights'] = opt_weights_transpose[0] 
opt_weights_transpose.reset_index(inplace=True)
opt_weights_transpose